In [ ]:
import pandas as pd
import psycopg2
import os

try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    print("Connected to PostgreSQL successfully!")
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")

In [ ]:
sql_query = """SELECT
    nu_ano,
    co_municipio_prova,
    nu_sequencial,
    no_municipio_prova,
    co_uf_prova,
    sg_uf_prova,
    nome_uf_prova,
    municipio_capital_uf_prova,
    regiao_codigo_prova,
    regiao_nome_prova,
    co_escola,
    co_uf_esc,
    sg_uf_esc,
    co_municipio_esc,
    no_municipio_esc,
    tp_localizacao_esc,
    tp_sit_func_esc,
    nota_cn_ciencias_da_natureza AS nota_cn, -- Simplificando nomes longos
    nota_ch_ciencias_humanas AS nota_ch,
    nota_lc_linguagens_e_codigos AS nota_lc,
    nota_mt_matematica AS nota_mt,
    nota_redacao,
    nota_media_5_notas,
    tp_lingua,
    tp_status_redacao,
    tp_dependencia_adm_esc,
    
    -- Coluna de Classificação da Rede de Ensino
    CASE tp_dependencia_adm_esc
       WHEN 'Estadual' THEN 'Pública'
       WHEN 'Federal' THEN 'Pública'
       WHEN 'Privada' THEN 'Privada'
       ELSE 'Não Classificado' 
    END AS Rede_Ensino_Classificada

FROM
    public.ed_enem_2024_resultados

WHERE 
    co_municipio_esc IN (
        '3104502', '3109303', '3109451', '3170404', 
        '5200100', '5300108', '5222203', '5200175', 
        '5200258', '5200308', '5200605', '5200803', 
        '5203203', '5222302', '5204003', '5205307', 
        '5205497', '5205513', '5205802', '5206206', 
        '5207907', '5208004', '5208608', '5213053', 
        '5214606', '5215231', '5215603', '5217302', 
        '5217609', '5219753', '5220009', '5220686', 
        '5221858'
    );
        """

df = pd.io.sql.read_sql_query(sql_query, conn)
# Now data is a pandas dataframe having the results of above query.
df.head()


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_preenchida = df.dropna()
df_preenchida.info()

In [ ]:
36496-28626

A perda de linhas foi de 7870

In [ ]:
df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
sns.histplot(df_preenchida['nota_media_5_notas'], bins=30, kde=True, ax=axes[0, 0])
sns.histplot(df_preenchida['nota_redacao'], bins=30, kde=True, ax=axes[0, 1])
sns.histplot(df_preenchida['nota_mt'], bins=30, kde=True, ax=axes[0, 2])
sns.histplot(df_preenchida['nota_lc'], bins=30, kde=True, ax=axes[1, 0])
sns.histplot(df_preenchida['nota_ch'], bins=30, kde=True, ax=axes[1, 1])
sns.histplot(df_preenchida['nota_cn'], bins=30, kde=True, ax=axes[1, 2])

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.countplot(x='no_municipio_esc', data=df_preenchida, ax=ax, hue='sg_uf_esc')

ax.set_xticklabels(ax.get_xticklabels(), rotation=75, horizontalalignment='right')
fig.tight_layout()


In [ ]:
df_preenchida["nu_ano"].value_counts()

In [ ]:
sns.countplot(x='tp_dependencia_adm_esc', data=df_preenchida, hue='sg_uf_esc')

In [ ]:
sns.countplot(x='tp_localizacao_esc', data=df_preenchida, hue='sg_uf_esc')

In [ ]:
df_preenchida.columns

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score




df_final = df_preenchida[['nota_cn', 'nota_ch', 'nota_lc', 'nota_mt', 'nota_redacao', 'rede_ensino_classificada']].copy()

# --- 2. PRÉ-PROCESSAMENTO E PREPARAÇÃO DOS DADOS ---

# 2.1. Variáveis Preditivas (X) e Variável Alvo (y)
X = df_final[['nota_cn', 'nota_ch', 'nota_lc', 'nota_mt', 'nota_redacao']]

# 2.2. Codificação da Variável Alvo: Privada=1, Pública=0
y = df_final['rede_ensino_classificada'].map({'Pública': 0, 'Privada': 1})

# 2.3. Divisão em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 2.4. Escalonamento (Standardization)
# A regressão logística se beneficia muito da normalização/escalonamento das variáveis de entrada.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# --- 3. CONSTRUÇÃO E TREINAMENTO DO MODELO LOGIT ---

# Inicializa o modelo de Regressão Logística (Logit)
# C=1.0 é o inverso da força de regularização (menor C = mais regularização).
model_logit = LogisticRegression(solver='liblinear', C=1.0, random_state=42)

# Treina o modelo
model_logit.fit(X_train_scaled, y_train)


# --- 4. AVALIAÇÃO DO MODELO ---

print("\n--- Resultados do Modelo Logit Frequentista ---")

# 4.1. Previsões
y_pred = model_logit.predict(X_test_scaled)
y_proba = model_logit.predict_proba(X_test_scaled)[:, 1]

# 4.2. Métricas de Desempenho
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Acurácia no Conjunto de Teste: {accuracy:.4f}")
print(f"AUC-ROC no Conjunto de Teste: {roc_auc:.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Pública (0)', 'Privada (1)']))


# --- 5. INTERPRETAÇÃO DOS COEFICIENTES (PESOS) ---

print("\n--- Coeficientes do Modelo Logit ---")

# Criamos uma série com os coeficientes e os nomes das features
coef_df = pd.Series(model_logit.coef_[0], index=X_train.columns)
intercept = model_logit.intercept_[0]

print(f"Intercepto (Log-Odds Base): {intercept:.4f}")
print("Pesos (Log-Odds) para as Notas Padronizadas:")
print(coef_df.to_string())

print("\n* Interpretação do Coeficiente (Exemplo):")
print(f"Para cada 1 desvio-padrão a mais em Matemática, a chance (odds) de ser de escola Privada (Y=1) aumenta por um fator de {np.exp(coef_df['nota_mt']):.3f} (mantendo a Redação constante).")

In [ ]:
import pandas as pd
import numpy as np
from cmdstanpy import CmdStanModel, install_cmdstan
from sklearn.preprocessing import StandardScaler # Import necessário
import os

# ... (Função create_simulated_df e carregamento de dados) ...
# (Assumindo que X_scaled, y, e df já estão definidos)

# --- 2. Geração do Modelo Stan (CORRIGIDO) ---

stan_code_fixed = """
// logit_rede_ensino.stan (Sintaxe Corrigida)

data {
  int<lower=0> N;          // Número total de observações
  int<lower=0> K;          // Número de preditores (notas)
  matrix[N, K] X;         // Matriz de preditores (notas padronizadas)
  
  // **CORREÇÃO AQUI:** array[N] int<lower=0, upper=1> y;
  array[N] int<lower=0, upper=1> y; // Variável resposta (0=Pública, 1=Privada)
}

parameters {
  vector[K] beta;           // Coeficientes das notas
  real beta0;               // Intercepto
}

model {
  // 1. Priors
  beta0 ~ normal(-1, 1.5); 
  beta ~ normal(0.5, 1.0);
  
  // 2. Likelihood
  vector[N] Z = beta0 + X * beta;
  y ~ bernoulli_logit(Z);
}

generated quantities {
    vector[N] p_privada_media = inv_logit(beta0 + X * beta);
}
"""

stan_file_path = 'logit_rede_ensino_fixed.stan'
with open(stan_file_path, 'w') as f:
    f.write(stan_code_fixed)

# ... (Instalação e Definição de stan_data) ...

# --- 4. EXECUÇÃO DA INFERÊNCIA BAYESIANA ---

print("Compilando o modelo Stan (Versão Corrigida)...")
# Compila o modelo Stan
model_stan = CmdStanModel(stan_file=stan_file_path) # Usando o novo caminho

print("Iniciando amostragem MCMC...")
# Executa o sampler MCMC (NUTS)
fit = model_stan.sample(
    data=stan_data, 
    chains=4, 
    iter_warmup=1000, 
    iter_sampling=2000, 
    seed=42
)

# ... (Análise dos resultados) ...

In [ ]:
print("\n--- Sumário Estatístico da Posterior (CmdStanPy) ---")
print(fit.summary())

# Para acessar e analisar os coeficientes (betas)
posterior_samples = fit.draws_pd()

# Mapeando os coeficientes para os nomes das disciplinas
betas_map = {
    'beta[1]': 'beta_Matemática',
    'beta[2]': 'beta_Redação',
    'beta[3]': 'beta_Linguagens'
}

# Exibindo o Intercepto e os Coeficientes
print("\n--- Medidas de Tendência Central (Posterior) ---")
print(f"Intercepto (beta0) Média: {posterior_samples['beta0'].mean():.4f}")
print("Coeficientes (Beta):")
for i, col in enumerate(notas_cols):
    stan_name = f'beta[{i+1}]'
    mean = posterior_samples[stan_name].mean()
    hdi_lower = np.percentile(posterior_samples[stan_name], 2.5)
    hdi_upper = np.percentile(posterior_samples[stan_name], 97.5)
    print(f"  {col}: Média = {mean:.4f}, 95% HDI = [{hdi_lower:.4f}, {hdi_upper:.4f}]")